# Observation simulation for Operations Rehearsal for Commissioning #4

In [ ]:
import copy
import numpy as np
import pandas as pd

from astropy.time import Time
from astropy.coordinates import Angle
from astropy import units as u

import matplotlib.pylab as plt

In [ ]:
# There ought to be a way to set the environment variable for this kernel
import os
os.environ["RUBIN_SIM_DATA_DIR"] = "/home/b/bechtol/rubin-user/rubin_sim_data"
os.getenv("RUBIN_SIM_DATA_DIR")

You can set environment variable in the kernel either by adding it to your .user_setups file (https://rsp.lsst.io/guides/notebooks/configuration/notebook-user-setups.html) if you installed rubin_sim into the standard kernel via pip,  or you can just symlink the actual directory to "~/rubin_sim_data" in the shell.

If you're using a non-standard kernel, you can set the env variable there too, but you would probably want to add a kernel-helper.sh file, sourced in your kernel.json file.  (more like https://confluence.lsstcorp.org/display/LSCO/Running+rubin_sim+and+the+LSST+stack+together+on+the+RSP)

In [ ]:
from rubin_scheduler.scheduler.model_observatory import ModelObservatory
from rubin_scheduler.scheduler.basis_functions import BaseBasisFunction, M5DiffBasisFunction, NotTwilightBasisFunction, NObsHighAmBasisFunction, FilterLoadedBasisFunction, OnceInNightBasisFunction, AvoidFastRevists
from rubin_scheduler.scheduler.detailers import DitherDetailer, CameraRotDetailer
from rubin_scheduler.scheduler.surveys import FieldSurvey
from rubin_scheduler.scheduler.schedulers import CoreScheduler
from rubin_scheduler.scheduler import  sim_runner
from rubin_scheduler.utils import ddf_locations
from rubin_scheduler.scheduler.utils import SchemaConverter
from rubin_scheduler.site_models import SeeingData

In [ ]:
# Note that can only set this date after downloading the full set sky brightness data
mjd_start = Time('2024-06-11 00:00:00.000', format='iso').mjd
mjd_start

In the cell below, turn off observatory downtime and clouds, but keep a realistic distribution of seeing

In [ ]:
# We need the start date of the survey, so let's load up our model observatory and get that from the conditions
nside = 32  # Specify the HEALpix resolution
mo = ModelObservatory(nside=nside, ideal_conditions=True, mjd_start=mjd_start)
mo.seeing_data = SeeingData(Time(mjd_start, format='mjd'))
conditions = mo.return_conditions()

In [ ]:
#conditions.mjd

In [ ]:
#conditions.mounted_filters = ['g', 'r', 'i']

In [ ]:
#conditions.mounted_filters

In [ ]:
#dir(conditions)

In [ ]:
#conditions.current_filter

In [ ]:
#conditions.night

In [ ]:
#conditions.mjd

In [ ]:
#conditions.moon_ra, conditions.moon_dec

In [ ]:
#conditions.rot_tel_pos

In [ ]:
#np.nanmax(conditions.airmass)

In [ ]:
#print(conditions.site.latitude_rad, conditions.site.longitude_rad)
#print(conditions.site.latitude, conditions.site.longitude)
#assert np.radians(conditions.site.longitude) == conditions.site.longitude_rad
#assert np.radians(conditions.site.latitude) == conditions.site.latitude_rad

## Field Selection

In [ ]:
"""
# Start with a set of fields chosen to be uniformly spaced in RA so that some field should be visible throughout the night all year
#ra_array = np.arange(0., 360., 30.)
ra_array = np.arange(0., 360., 60.) + 5
dec_array = np.tile(-25., len(ra_array))
#dec_array = np.tile(-30., len(ra_array))
print(ra_array)
print(dec_array)
"""

In [ ]:
"""
targets = (
    (95., -25.), # High stellar densty, low extinction
    (125., -15.), # High stellar densty, low extinction
    #(360 - 145., -25.),
    (216, -12.5), # DEEP Solar Systen
    (360 - 110., 0.), # High stellar densty, low extinction
    #(219.80, -0.600), # DESI, GAMA, HSC DR2, KiDS-N
    #(360 - 53., -25),
    (270.891667, -30.033889), # Baade's Window
    (310, -19), # DEEP Solar System
    (9.45, -44.0),
    (35.708333, -4.75),
    (53.125, -28.1), # ECDFS
    (150.1, 2.1819444444444445), # COSMOS
    (58.9, -49.315), # EDFS_a
    (63.6, -47.6), # EDFS_b
)
"""
"""
# 2 January 2024
targets = (
    (95., -25.), # High stellar densty, low extinction
    (125., -15.), # High stellar densty, low extinction
    (179.60, 0.000), # DESI, GAMA, HSC DR2, KiDS-N
    #(360 - 145., -25.),
    (360 - 135., -40.), # High stellar densty, low extinction
    (216, -12.5), # DEEP Solar Systen
    (360 - 110., 2.), # High stellar densty, low extinction
    #(219.80, -0.600), # DESI, GAMA, HSC DR2, KiDS-N
    #(360 - 53., -25),
    #(270.891667, -30.033889), # Baade's Window
    (300., -41.), # High stellar densty, low extinction 
    (280., -48.), # High stellar densty, low extinction 
    (310, -19), # DEEP Solar System
    (9.45, -44.0), # ELAIS-S1 LSST DDF
    (35.708333, -4.75), # LSST DDF
    (53.125, -28.1), # ECDFS
    (150.1, 2.1819444444444445), # COSMOS
    (58.9, -49.315), # EDFS_a
    (63.6, -47.6), # EDFS_b
)


ra_array, dec_array = zip(*targets)
print(ra_array)
print(dec_array)
"""

In [ ]:
"""
ra_array = []
dec_array = []
for key in ['ELAISS1', 'XMM_LSS', 'ECDFS', 'COSMOS', 'EDFS_a']:
    ra_array.append(ddf_locations[key][0])
    dec_array.append(ddf_locations[key][1])

for key in ['A0', 'B0']:
    ra_array.append(deep_locations[key][0])
    dec_array.append(deep_locations[key][1])
"""

In [ ]:
"""
fields = {}
for ra, dec in zip(ra_array, dec_array):
    name = f"{int(ra)}{int(dec)}"
    fields[name] = {"RA": ra, "dec": dec}
print(fields)
"""

In [ ]:
import ops_rehearsal_fields_or4 as ops_rehearsal_fields
import importlib
importlib.reload(ops_rehearsal_fields)

name_fields, ra_fields, dec_fields = zip(*ops_rehearsal_fields.fields)

fields = {}
for name, ra, dec in zip(name_fields, ra_fields, dec_fields):
    fields[name] = {"RA": ra, "dec": dec}
for key in fields:
    print(key, fields[key]["RA"], fields[key]["dec"])

In [ ]:
import healpy as hp

In [ ]:
m_stellar_density = np.load('/home/b/bechtol/rubin_sim_data/maps/StarMaps/starDensity_g_nside_64.npz')

In [ ]:
hp.mollview(np.log10(m_stellar_density['starDensity'][:,20]), title='log10(stellar density)')
hp.projscatter(ra_fields, dec_fields, lonlat=True, color='red')

In [ ]:
#m_extinction = np.load('/home/b/bechtol/rubin_sim_data/maps/DustMaps/dust_nside_32.npz')

In [ ]:
#hp.mollview(m_extinction['ebvMap'], min=0, max=0.2, title='E(B-V)')
#hp.projscatter(ra_fields, dec_fields, lonlat=True, color='red')

## Basis Functions

In [ ]:
from rubin_scheduler.scheduler import features
from rubin_scheduler.scheduler.utils import IntRounded

class AvoidFastRevisitsBasisFunction(BaseBasisFunction):
    """Marks targets as unseen if they are in a specified time window
    in order to avoid fast revisits.

    Parameters
    ----------
    filtername: `str` ('r')
        The name of the filter for this target map.
    gap_min : `float` (25.)
        Minimum time for the gap (minutes).
    nside: `int` (default_nside)
        The healpix resolution.
    penalty_val : `float` (np.nan)
        The reward value to use for regions to penalize.
        Will be masked if set to np.nan (default).
    """

    def __init__(self, survey_name=None, gap_min=30.0, penalty_val=np.nan):
        super(AvoidFastRevisitsBasisFunction, self).__init__()

        self.penalty_val = penalty_val

        self.gap_min = IntRounded(gap_min / 60.0 / 24.0)

        self.survey_features = dict()
        self.survey_features["LastObservation"] = features.LastObservation(survey_name)

    def _calc_value(self, conditions, indx=None):
        diff = IntRounded(conditions.mjd - self.survey_features["LastObservation"].feature['mjd'])
        if diff < self.gap_min:
            result = self.penalty_val
        else:
            result = 0.
        return result

In [ ]:
#bfs = AvoidFastRevistsBasisFunction(survey_name='test')
#conditions.mjd - bfs.survey_features["LastObservation"].feature['mjd'][0]

In [ ]:
class RisingBasisFunction(BaseBasisFunction):
    """Reward fields that are rising.

    Parameters
    ----------
    RA : float
        The RA of the point in the sky (degrees)
    """

    def __init__(self, RA, weight=0.1):
        super(RisingBasisFunction, self).__init__()
        self.ra_hours = RA * 24 / 360.0
        self.weight = weight

    # Probably not needed
    def check_feasibility(self, conditions):
        result = True
        return result

    def _calc_value(self, conditions, indx=None):
        hour_angle = conditions.lmst - self.ra_hours
        return -1 * self.weight * hour_angle

In [ ]:
#from astropy.coordinates import SkyCoord, angular_separation

In [ ]:
#ra_1, dec_1 = 180., 0.
#ra_2, dec_2 = 181., 1.

#angsep = angular_separation(np.radians(ra_1), np.radians(dec_1), np.radians(ra_2), np.radians(dec_2))
#print(np.degrees(angsep))

In [ ]:
from astropy.coordinates import angular_separation

class AvoidMoonBasisFunction(BaseBasisFunction):
    """Only execute if angular separation to the Moon is above a specified threshold.

    Parameters
    ----------
    ra : float
        The RA of the point in the sky (degrees)
    dec : float
        The declination of the point in the sky (degrees)
    min_moon_dist : float
        The minimum angular separation to the moon (degrees)
    """

    def __init__(self, ra, dec, min_moon_dist=20.):
        super(AvoidMoonBasisFunction, self).__init__()
        self.ra = ra
        self.dec = dec
        self.min_moon_dist = min_moon_dist

    def check_feasibility(self, conditions):
        moon_dist = np.degrees(
            angular_separation(
                np.radians(self.ra), np.radians(self.dec), conditions.moon_ra, conditions.moon_dec
            )
        )
        if moon_dist > self.min_moon_dist:
            result = True
        else:
            result = False
        return result

In [ ]:
from rubin_scheduler.utils import approx_ra_dec2_alt_az

class MaximumAirmassBasisFunction(BaseBasisFunction):
    """Only execute if the airmass is a below a specified threshold.

    Parameters
    ----------
    ra : float
        The RA of the point in the sky (degrees)
    dec : float
        The declination of the point in the sky (degrees)
    max_airmass : float
        The maximum airmass
    """

    def __init__(self, ra, dec, max_airmass=2.0):
        super(MaximumAirmassBasisFunction, self).__init__()
        self.ra = ra
        self.dec = dec
        self.max_airmass = max_airmass

    def check_feasibility(self, conditions):
        alt, az = approx_ra_dec2_alt_az(
            self.ra,
            self.dec,
            conditions.site.latitude,
            conditions.site.longitude,
            conditions.mjd,
        )
        
        airmass = 1. / np.cos(np.radians(90. - alt))
        
        if airmass < self.max_airmass:
            result = True
        else:
            result = False

        #import pdb; pdb.set_trace()
        
        return result

In [ ]:
class AirmassRangeBasisFunction(BaseBasisFunction):
    """Only execute if the airmass is within a specified range at the start and end 
    of the observation sequence.

    Parameters
    ----------
    ra : float
        The RA of the point in the sky (degrees)
    dec : float
        The declination of the point in the sky (degrees)
    min_airmass : float
        The minimum airmass
    max_airmass : float
        The maximum airmass
    duration : float
        The duration (days)
    """

    def __init__(self, ra, dec, min_airmass=1.0, max_airmass=2.0, duration=0.0275):
        super(AirmassRangeBasisFunction, self).__init__()
        self.ra = ra
        self.dec = dec
        self.min_airmass = min_airmass
        self.max_airmass = max_airmass
        self.duration = duration

    def check_feasibility(self, conditions):
        alt_start, az_start = approx_ra_dec2_alt_az(
            self.ra,
            self.dec,
            conditions.site.latitude,
            conditions.site.longitude,
            conditions.mjd,
        )
        
        airmass_start = 1. / np.cos(np.radians(90. - alt_start))

        alt_end, az_end = approx_ra_dec2_alt_az(
            self.ra,
            self.dec,
            conditions.site.latitude,
            conditions.site.longitude,
            conditions.mjd + self.duration,
        )
        
        airmass_end = 1. / np.cos(np.radians(90. - alt_end))        

        if (airmass_start > self.min_airmass) \
            and (airmass_start < self.max_airmass) \
            and (airmass_end > self.min_airmass) \
            and (airmass_end < self.max_airmass):
            result = True
        else:
            result = False
        
        return result

In [ ]:
from rubin_scheduler.scheduler.detailers import BaseDetailer
from rubin_scheduler.utils import _approx_ra_dec2_alt_az, _approx_altaz2pa

class CameraRotPerObservationListDetailer(BaseDetailer):
    """
    Randomly set the camera rotation for each observation list.

    Parameters
    ----------
    max_rot : `float` (90.)
        The maximum amount to offset the camera (degrees)
    min_rot : `float` (90)
        The minimum to offset the camera (degrees)
    """

    def __init__(self, max_rot=90.0, min_rot=-90.0, seed=42, per_visit_rot=0.): # nnights=7305):
        self.survey_features = {}

        self.current_night = -1
        self.max_rot = np.radians(max_rot)
        self.min_rot = np.radians(min_rot)
        self.range = self.max_rot - self.min_rot
        self.seed = seed
        self.per_visit_rot = per_visit_rot
        #self.rng = np.random.default_rng(seed)
        #self.offsets = self.rng.random(100 * nnights)
        self.offset = None

    def _generate_offsets_orig(self, n_offsets, night, mjd):
        #print(mjd)
        #print(np.asarray(mjd).item() % 100)
        mjd_hash = round(100 * (np.asarray(mjd).item() % 100))
        #print(mjd_hash)
        rng = np.random.default_rng(mjd_hash * self.seed)
        #print(night)
        #print(type(night))
        #print(mjd)
        #print(type(mjd))
        #index = 1
        #index = round(
        #    (100 * night) + (100. * (mjd % 100))
        #)
        #self.offset = self.offsets[index] * self.range + self.min_rot
        self.offset = (rng.random() * self.range) + self.min_rot
        offsets = np.ones(n_offsets) * self.offset
        return offsets

    def _generate_offsets_filter_change(self, filter_list, mjd, initial_offset):
        """Generate a random camera rotation for each filter change.
        """
        
        mjd_hash = round(100 * (np.asarray(mjd).item() % 100))
        rng = np.random.default_rng(mjd_hash * self.seed)
        
        offsets = np.zeros(len(filter_list))
        offset = np.asarray(initial_offset).item()
        offsets[0] = offset
        
        for ii in range(1, len(offsets)):            
            if filter_list[ii] != filter_list[ii - 1]:
                # Filter change
                offset = (rng.random() * self.range) + self.min_rot
            else:
                # If no filter change, apply a small a small rotation
                offset += np.radians(self.per_visit_rot)
            offsets[ii] = offset

        return offsets
        
    def __call__(self, observation_list, conditions):
        # Generate offsets in camera rotator
        #offsets = self._generate_offsets_orig(len(observation_list), conditions.night, conditions.mjd)
        #import pdb; pdb.set_trace()
        #offsets = self._generate_offsets(observation_list, conditions.night, conditions.mjd)

        filter_list = [np.asarray(obs['filter']).item() for obs in observation_list]
        offsets = self._generate_offsets_filter_change(filter_list, conditions.mjd, conditions.rot_tel_pos)
        
        for i, obs in enumerate(observation_list):
            alt, az = _approx_ra_dec2_alt_az(
                obs["RA"],
                obs["dec"],
                conditions.site.latitude_rad,
                conditions.site.longitude_rad,
                conditions.mjd,
            )
            obs_pa = _approx_altaz2pa(alt, az, conditions.site.latitude_rad)
            obs["rotSkyPos"] = (offsets[i] - obs_pa) % (2.0 * np.pi)
            obs["rotTelPos"] = offsets[i]

        return observation_list

In [ ]:
# Testing non-repeatability of simulations

In [ ]:
#filter_list = ['g', 'g', 'r', 'r', 'i', 'i']
#mjd = [0.001, 0.002, 0.003, 0.004]
#initial_offset = 45.

#detailer = CameraRotPerObservationListDetailer()
#detailer._generate_offsets_filter_change(filter_list, conditions.mjd, conditions.rot_tel_pos)

In [ ]:
detailer = DitherDetailer(max_dither=0.2, per_night=False, seed=42)
detailer._generate_offsets(10, 2)

In [ ]:
from rubin_scheduler.scheduler.utils import wrap_ra_dec
from rubin_scheduler.utils import (
    gnomonic_project_tosky,
)

class DitherDetailerRepeatable(BaseDetailer):
    """
    make a uniform dither pattern. Offset by a maximum radius in a random
    direction. Mostly intended for DDF pointings, the BaseMarkovDF_survey
    class includes dithering for large areas.

    Parameters
    ----------
    max_dither : `float` (0.7)
        The maximum dither size to use (degrees).
    per_night : `bool` (True)
        If true, us the same dither offset for an entire night
    nnights : `int` (7305)
        The number of nights to pre-generate random dithers for


    """

    def __init__(self, max_dither=0.7, seed=42, per_night=True, nnights=7305):
        self.survey_features = {}

        self.current_night = -1
        self.max_dither = np.radians(max_dither)
        self.per_night = per_night
        self.rng = np.random.default_rng(seed)
        self.angles = self.rng.random(nnights) * 2 * np.pi
        self.radii = self.max_dither * np.sqrt(self.rng.random(nnights))
        self.offsets = (self.rng.random((nnights, 2)) - 0.5) * 2.0 * self.max_dither
        self.offset = None

    def _generate_offsets(self, n_offsets, night):
        if self.per_night:
            if night != self.current_night:
                self.current_night = night
                self.offset = self.offsets[night, :]
                angle = self.angles[night]
                radius = self.radii[night]
                self.offset = np.array([radius * np.cos(angle), radius * np.sin(angle)])
            offsets = np.tile(self.offset, (n_offsets, 1))
        else:
            #self.rng = np.random.default_rng() # REMOVE THIS LINE
            angle = self.rng.random(n_offsets) * 2 * np.pi
            radius = self.max_dither * np.sqrt(self.rng.random(n_offsets))
            offsets = np.array([radius * np.cos(angle), radius * np.sin(angle)]).T

        return offsets

    def __call__(self, observation_list, conditions):
        if len(observation_list) == 0:
            return observation_list
        # Generate offsets in RA and Dec
        offsets = self._generate_offsets(len(observation_list), conditions.night)

        obs_array = np.concatenate(observation_list)
        new_ra, new_dec = gnomonic_project_tosky(
            offsets[:, 0], offsets[:, 1], obs_array["RA"], obs_array["dec"]
        )
        new_ra, new_dec = wrap_ra_dec(new_ra, new_dec)
        for i, obs in enumerate(observation_list):
            observation_list[i]["RA"] = new_ra[i]
            observation_list[i]["dec"] = new_dec[i]
        return observation_list

In [ ]:
detailer = DitherDetailerRepeatable(max_dither=0.2, per_night=False, seed=42)
detailer._generate_offsets(10, 2)

In [ ]:
detailer._generate_offsets(10, 2)

## Scheduler

In [ ]:
# Basis functions
#bfs = []

sun_alt_limit = -12.0 # deg

bfs = [
    M5DiffBasisFunction(filtername='r', nside=nside),
    NotTwilightBasisFunction(sun_alt_limit=sun_alt_limit),
    #AvoidFastRevistsBasisFunction(filtername='r', nside=nside, gap_min=25.0, penalty_val=np.nan),
    #FilterLoadedBasisFunction(filternames=['g', 'r', 'i']),
    #OnceInNightBasisFunction(notes=['dense_dithered']),
    #NObsHighAmBasisFunction(),
]

In [ ]:
#bfs[2].check_feasibility(conditions)

In [ ]:
#rng = np.random.default_rng(42)
#for ii in range(0, 10):
#    print(rng.random())

In [ ]:
# Add in detailer
#detailers = None # Test repeatability
detailers = [
    #DitherDetailer(max_dither=0.2, per_night=False),
    DitherDetailerRepeatable(max_dither=0.2, per_night=False),
    CameraRotPerObservationListDetailer(
        max_rot=45.0, 
        min_rot=-45.0, 
        seed=42, 
        per_visit_rot=1.,
    ),
    #CameraRotDetailer(per_night=True),
]

In [ ]:
nvis_master = [20, 20, 20]
sequence = "gri"
exptime = 30
u_exptime = 30
reward_value = None
nexp = 1 # 1 --> single 30 second exposure

min_moon_dist = 20. # deg

params = {
    "low_airmass": {
        "min_airmass": 1.0,
        "max_airmass": 1.6,
        "gap_min": 120.
    },
    "high_airmass": {
        "min_airmass": 1.3,
        "max_airmass": 2.1,
        "gap_min": 30.
    },  
}

surveys = []
for field_name in fields.keys():

    print(field_name)
    
    # Low airmass surveys
    survey_name_low_airmass = field_name + "_low_airmass"
    #min_airmass = 1.0
    #max_airmass = 1.6
    ##gap_min = 30.
    #gap_min = 120. # 12 hr
    bfs_survey_low_airmass = copy.deepcopy(bfs)
    bfs_survey_low_airmass.append(
        RisingBasisFunction(RA=fields[field_name]["RA"])
    )
    bfs_survey_low_airmass.append(
        AvoidFastRevisitsBasisFunction(
            survey_name=survey_name_low_airmass, 
            gap_min=params["low_airmass"]["gap_min"]
        )
    )
    bfs_survey_low_airmass.append(AvoidMoonBasisFunction(
        ra=fields[field_name]["RA"], 
        dec=fields[field_name]["dec"], 
        min_moon_dist=min_moon_dist)
    )
    bfs_survey_low_airmass.append(
        AirmassRangeBasisFunction(
            ra=fields[field_name]["RA"], 
            dec=fields[field_name]["dec"],
            min_airmass=params["low_airmass"]["min_airmass"],
            max_airmass=params["low_airmass"]["max_airmass"]
        )
    )
    
    surveys.append(
        FieldSurvey(
            bfs_survey_low_airmass,
            fields[field_name]["RA"],
            fields[field_name]["dec"],
            sequence=sequence,
            nvis=nvis_master,
            exptime=exptime,
            u_exptime=u_exptime,
            survey_name=survey_name_low_airmass,
            reward_value=reward_value,
            nside=nside,
            nexp=nexp,
            detailers=detailers,
        )
    )

    
    # High airmass survey
    survey_name_high_airmass = field_name + "_high_airmass"
    #min_airmass = 1.3
    #max_airmass = 2.1
    #gap_min = 30. # 0.5 hr
    bfs_survey_high_airmass = copy.deepcopy(bfs)
    bfs_survey_high_airmass.append(
        RisingBasisFunction(RA=fields[field_name]["RA"])
    )
    bfs_survey_high_airmass.append(
        AvoidFastRevisitsBasisFunction(
            survey_name=survey_name_high_airmass, 
            gap_min=params["high_airmass"]["gap_min"]
        )
    )
    bfs_survey_high_airmass.append(AvoidMoonBasisFunction(
        ra=fields[field_name]["RA"], 
        dec=fields[field_name]["dec"], 
        min_moon_dist=min_moon_dist)
    )
    bfs_survey_high_airmass.append(AirmassRangeBasisFunction(
        ra=fields[field_name]["RA"], 
        dec=fields[field_name]["dec"],
        min_airmass=params["high_airmass"]["min_airmass"],
        max_airmass=params["high_airmass"]["max_airmass"])
    )

    surveys.append(
        FieldSurvey(
            bfs_survey_high_airmass,
            fields[field_name]["RA"],
            fields[field_name]["dec"],
            sequence=sequence,
            nvis=nvis_master,
            exptime=exptime,
            u_exptime=u_exptime,
            survey_name=survey_name_high_airmass,
            reward_value=reward_value,
            nside=nside,
            nexp=nexp,
            detailers=detailers,
        )
    )


In [ ]:
#surveys[0].basis_functions

In [ ]:
#scheduler = CoreScheduler(surveys, nside=nside)

In [ ]:
scheduler = CoreScheduler(surveys, nside=nside)
# Note the deep copy used here. The function sim_runner modifies the observatory
#new_mo, new_scheduler, observations = sim_runner(copy.deepcopy(mo), copy.deepcopy(scheduler), survey_length=3., verbose=True)
new_mo, new_scheduler, observations = sim_runner(copy.deepcopy(mo), copy.deepcopy(scheduler), survey_length=21., verbose=True)
del new_mo
del new_scheduler

In [ ]:
df = pd.DataFrame(np.hstack(observations))
df

In [ ]:
print(df['RA'][0])
print(df['mjd'][0])

In [ ]:
#np.testing.assert_almost_equal(df['RA'].iloc[-1], 2.1821887017065236)

In [ ]:
#np.testing.assert_almost_equal(df['RA'].iloc[-1], 2.1812680654192746)

In [ ]:
# 2.179457
# 60390.002539 	

In [ ]:
df.columns

In [ ]:
# Observability
assert np.all(observations['airmass'] < params["high_airmass"]["max_airmass"])
moon_dist_margin = 0.5 * (hp.nside2resol(nside)*u.rad).to(u.deg).value
assert np.all(np.degrees(observations['moonDist']) > (min_moon_dist - moon_dist_margin))

In [ ]:
# Forbid 2 x 15 sec snaps
assert np.all(observations['nexp'] == 1)

In [ ]:
print(np.sum(observations['airmass'] > params["high_airmass"]["max_airmass"]))
print(np.max(observations['airmass']))

In [ ]:
print(np.min(np.degrees(observations['moonDist'])))

In [ ]:
np.unique(observations['note'])

In [ ]:
np.unique(observations['filter'])

In [ ]:
assert np.std(observations['FWHM_500']) > 0.1

In [ ]:
print(np.std(observations['FWHM_500']))

In [ ]:
print(
    Time(np.min(observations['mjd']), format='mjd').iso
)
print(
    Time(np.max(observations['mjd']), format='mjd').iso
)

## Rotation Angle Check

"I'm guessing that OpSim starts with RotSkyPos and then computes RotTelPos?  So we want to keep RSP but change RTP in the OpSim outputs.  I think the bit that was missing in OR3 is the -270 degrees."

Anticipate that will need to apply a correction to one of the columns.

In [ ]:
"""
def rtp2rsp(rtp, pa):
    results = [Angle(270*u.deg + _rtp*u.deg - _pa*u.deg).wrap_at(180*u.deg).value for (_rtp, _pa) in zip(rtp, pa)]
    return np.array(results)

def rsp2rtp(rsp, pa):
    results = [Angle(rsp*u.deg + pa*u.deg - 270*u.deg).wrap_at(180*u.deg).value for (_rsp, _pa) in zip(rsp, pa)]
    return np.array(results)
"""

In [ ]:
"""
rsp = rtp2rsp(
    np.degrees(observations['rotTelPos']),
    np.degrees(observations['pa']),
)
rsp - np.degrees(observations['rotSkyPos'])
"""

In [ ]:
"""
rtp = rsp2rtp(
    np.degrees(observations['rotSkyPos']),
    np.degrees(observations['pa']),
)
rtp - np.degrees(observations['rotTelPos'])
"""

## Apply rotSkyPos Update

See the following notebook for examples of updating `rotSkyPos` values.

https://github.com/rhiannonlynne/notebooks/blob/main/UpdateRotSkyPos.ipynb

In [ ]:
"""
from rubin_scheduler.utils import Site

from astropy.coordinates import SkyCoord
import astropy.units as u

from astroplan import Observer, FixedTarget
"""

In [ ]:
"""
lsst = Site('LSST')
observer = Observer(longitude=lsst.longitude*u.deg, latitude=lsst.latitude*u.deg,
                    elevation=lsst.height*u.m, name="LSST", 
                    timezone="Chile/Continental", 
                    pressure=lsst.pressure*u.mBa, 
                    temperature=lsst.temperature*u.deg_C, 
                    relative_humidity=lsst.humidity)
"""

In [ ]:
"""
print('LSST site')
print('lat', lsst.latitude*u.deg)
print('lon', lsst.longitude*u.deg)
print('elevation', lsst.height*u.m)
print('pressure', lsst.pressure*u.mBa)
print('temperature', lsst.temperature*u.deg_C)
print('humidity', lsst.humidity)
"""

In [ ]:
"""
df[['RA', 'dec', 'mjd', 'rotTelPos', 'pa']]
"""

In [ ]:
"""
coords = SkyCoord(
    np.degrees(df['RA'].values), 
    np.degrees(df['dec'].values), 
    unit='deg', 
    frame='icrs'
)
targets = FixedTarget(coords)
times = Time(df['mjd'], format='mjd', scale='tai')
altaz = observer.altaz(times, targets)
pa = observer.parallactic_angle(times, targets, kind='apparent')
pa = pa.deg % 360
rotSkyPos = (pa - np.degrees(df['rotTelPos'].values) - 90) % 360
"""

In [ ]:
"""
rotSkyPos
"""

In [ ]:
"""
plt.figure()
plt.scatter(np.degrees(df['rotSkyPos']), rotSkyPos)
"""

In [ ]:
"""
df['pa'] = pa
df['rotSkyPos'] = rotSkyPos
"""

# Apply rotSkyPos Update from SMTN-019

https://smtn-019.lsst.io/v/DM-44258/index.html

In [ ]:
from rotation_angles import pseudo_parallactic_angle, rtp_to_rsp

In [ ]:
rotTelPos = np.degrees(df['rotTelPos'].values)
ra = np.degrees(df['RA'].values)
dec = np.degrees(df['dec'].values)
mjd = Time(df['mjd'], format='mjd', scale='utc').tai.mjd

In [ ]:
rotSkyPos = rtp_to_rsp(rotTelPos, ra, dec, mjd)

In [ ]:
plt.figure()
plt.scatter(np.degrees(df['rotSkyPos']), rotSkyPos)

In [ ]:
# Pseudo parallactic angle
ppa = pseudo_parallactic_angle(ra, dec, mjd)

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u

from astroplan import Observer

coord = SkyCoord(ra*u.deg, dec*u.deg)
observer= Observer.at_site("LSST")
obstime = Time(mjd, format='mjd', scale='tai')
pa = observer.parallactic_angle(obstime, coord).deg

In [ ]:
pa

In [ ]:
plt.figure()
plt.scatter(pa, ppa - pa)
plt.ylim(-2, 0)

In [ ]:
plt.figure()
plt.scatter(np.degrees(df['pa']), pa)

In [ ]:
df['pa'] = np.radians(pa)
df['rotSkyPos'] = np.radians(rotSkyPos)

In [ ]:
print(np.min(df['pa']))
print(np.max(df['pa']))
print(np.min(df['rotSkyPos']))
print(np.max(df['rotSkyPos']))

## Quick Evaluation of Observation Simulation

In [ ]:
f2c = {'u': 'purple', 'g': 'blue', 'r': 'green',
       'i': 'cyan', 'z': 'orange', 'y': 'red'}

plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(observations['mjd'][in_filt], np.degrees(observations['alt'][in_filt]), 
                 'o', markersize=1, color=f2c[filtername], label=filtername)
plt.legend()
plt.xlabel('MJD')
plt.ylabel('Altitude (degrees)')

In [ ]:
f2c = {'u': 'purple', 'g': 'blue', 'r': 'green',
       'i': 'cyan', 'z': 'orange', 'y': 'red'}

plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(observations['mjd'][in_filt], observations['airmass'][in_filt], 
                 'o', markersize=1, color=f2c[filtername], label=filtername)
plt.legend()
plt.xlabel('MJD')
plt.ylabel('Airmass')

In [ ]:
plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(observations['mjd'][in_filt], observations['slewtime'][in_filt], 
                 'o', markersize=1, color=f2c[filtername], label=filtername)
plt.xlim(observations['mjd'][0], observations['mjd'][500])

In [ ]:
plt.figure(dpi=200)
plt.scatter(observations['mjd'], np.degrees(observations['rotSkyPos']), c=np.degrees(observations['RA']), s=1)
plt.colorbar(label='RA (deg)')

plt.xlabel('MJD')
plt.ylabel('rotSkyPos')

In [ ]:
f2c = {'u': 'purple', 'g': 'blue', 'r': 'green',
       'i': 'cyan', 'z': 'orange', 'y': 'red'}

plt.figure(dpi=200)
#plt.scatter(observations['mjd'], np.degrees(observations['rotTelPos']), c=np.degrees(observations['RA']), s=1)
#plt.colorbar(label='RA (deg)')

for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(observations['mjd'][in_filt], np.degrees(observations['rotTelPos'][in_filt]), 
                 'o', markersize=1, color=f2c[filtername], label=filtername)
plt.legend()

plt.xlabel('MJD')
plt.ylabel('rotTelPos')
plt.xlim(observations['mjd'][0], observations['mjd'][500])

In [ ]:
plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(np.degrees(observations['RA'][in_filt]), observations['airmass'][in_filt], 
                 'o', markersize=1, color=f2c[filtername], label=filtername)

plt.legend()
plt.xlabel('RA (deg)')
plt.ylabel('Airmass')

In [ ]:
plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(observations['mjd'][in_filt], np.degrees(observations['RA'][in_filt]), 
                 'o', markersize=1, color=f2c[filtername], label=filtername)

plt.legend()
plt.xlabel('MJD')
plt.ylabel('RA (deg)')

In [ ]:
plt.figure(dpi=200)
#for filtername in f2c:
#    in_filt = np.where(observations['filter'] == filtername)[0]
#    if in_filt.size > 0:
#        plt.plot(observations['mjd'][in_filt], np.degrees(observations['moonDist'][in_filt]), 
#                 'o', markersize=1, color=f2c[filtername], label=filtername)

plt.scatter(observations['mjd'], np.degrees(observations['moonDist']), c=observations['moonPhase'], s=1)
plt.colorbar(label='moonPhase')

#plt.legend()
plt.xlabel('MJD')
plt.ylabel('moonDist (deg)')

In [ ]:
plt.figure(dpi=200)
for filtername in f2c:
    in_filt = np.where(observations['filter'] == filtername)[0]
    if in_filt.size > 0:
        plt.plot(np.degrees(observations['RA'][in_filt]), np.degrees(observations['rotSkyPos'][in_filt]), 
                 'o', markersize=1, color=f2c[filtername], label=filtername)

plt.legend()
plt.xlabel('RA (deg)')
plt.ylabel('rotSkyPos')

In [ ]:
gap = (np.diff(observations['mjd']) * 24. * 3600.)

plt.figure()
plt.yscale('log')
bins = np.linspace(0., 300., 101)
plt.hist(gap, bins=bins)
plt.ylabel('Counts')
plt.xlabel('Time Gap (s)')

selection = gap > 300.
for index in np.nonzero(selection)[0]:
    print(gap[index], observations['mjd'][index + 1])

## Write the output

In [ ]:
#import sqlite3

In [ ]:
# open up a connection to a new database
#conn = sqlite3.connect('ops_rehearsal_apr_2024.db')

In [ ]:
#df.to_sql('observations', conn, index=False, if_exists='replace')

For ease of use downstream, make the pairs of high-airmass and low-airmass surveys appear as a single conceptual survey in the opSim database.

In [ ]:
def stripArray(a, string):
    return np.array([_.strip(string) for _  in a])

In [ ]:
# Request to fill the target column with the field name
observations['target'] = stripArray(stripArray(observations['note'], "_low_airmass"), "_high_airmass")

In [ ]:
observations['note'] = observations['target']

In [ ]:
assert np.all(observations['target'] == observations['note'])

Convert observation array to the standard opsim schema

Note that the SchemaConverter is only changing the column names and changing angles between radians and decimal degrees, but is not doing other manipulation of sky and telescope rotations.

In [ ]:
filename = 'ops_rehearsal_jun_2024_v3.db'
#filename = 'ops_rehearsal_jun_2024_v3_repeatability_test.db'
SchemaConverter().obs2opsim(observations, filename=filename, info=None, delete_past=True)

In [ ]:
# Check Consistency with Previous Runs

In [ ]:
#conn_new = sqlite3.connect('ops_rehearsal_apr_2024.db')
#observations_new = pd.read_sql('select * from observations;', conn_new)

#conn_old = sqlite3.connect('ops_rehearsal_apr_2024.db.BACKUP')
#conn_old = sqlite3.connect('/sdf/data/rubin/shared/ops-rehearsal-3/scheduler_sims/ops_rehearsal_apr_2024.db')
#observations_old = pd.read_sql('select * from observations;', conn_old)

In [ ]:
#observations_new

In [ ]:
#observations_old

In [ ]:
#observations_new['RA'].values == observations_old['RA'].values

In [ ]:
import sqlite3

In [ ]:
conn_new = sqlite3.connect(filename)
observations_new = pd.read_sql('select * from observations;', conn_new)

In [ ]:
observations_new.columns

In [ ]:
observations_new['target']

In [ ]:
assert np.all(observations_new['numExposures'] == 1)

## Repeatability Test

In [ ]:
filename_1 = 'ops_rehearsal_jun_2024_v3.db'
filename_2 = 'ops_rehearsal_jun_2024_v3_repeatability_test.db'

conn_1 = sqlite3.connect(filename_1)
observations_1 = pd.read_sql('select * from observations;', conn_1)

conn_2 = sqlite3.connect(filename_2)
observations_2 = pd.read_sql('select * from observations;', conn_2)

assert np.all(observations_1 == observations_2)